In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Image Retrieval'
!ls

/content/drive/MyDrive/Image Retrieval
 augmentation.ipynb
 binary_scenario
 binary_scenario_augmented_400X
 checkpoint
 citra_split.ipynb
 comparing_graph.ipynb
 comparing_retrieval.ipynb
 core
'data binary splitting.ipynb'
'data subclass splitting.ipynb'
 extract_feature_binary.ipynb
 extract_feature_subclass.ipynb
 original
 paper
 reconstruction
 retrieval_1_100X.ipynb
 retrieval_1_100X_subclass.ipynb
 retrieval_1_200X.ipynb
 retrieval_1_200X_subclass.ipynb
 retrieval_1_400X.ipynb
 retrieval_1_400X_subclass.ipynb
 subclass_400
 subclass_scenario
 train_auto_encoder_magnification_1_100.ipynb
 train_auto_encoder_magnification_1_200.ipynb
 train_auto_encoder_magnification_1_400.ipynb
 train_auto_encoder_magnification_1_40.ipynb
 train_auto_encoder_subclass_1_100.ipynb
 train_auto_encoder_subclass_1_200.ipynb
 train_auto_encoder_subclass_1_400.ipynb
 train_auto_encoder_subclass_1_40.ipynb
 training_1_100
 training_1_100.h5
 training_1_100.json
 training_1_100_subclass
 training_1_100_s

In [3]:
import matplotlib
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint 
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import json

In [4]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

In [5]:
class ConvAutoEncoder:
    @staticmethod
    def build(width, height, depth, filters=(128,), latent_dim=48):
        input_shape = (height, width, depth)
        channel_dim = -1
        inputs = Input(shape=input_shape)
        x = inputs
        # Encoder layer
        for f in filters:
            x = Conv2D(f, (3, 3), strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=channel_dim, name='enc_filter_'+str(f))(x)

        volume_size = K.int_shape(x)
        x = Flatten()(x)
        # Latent layer
        latent = Dense(latent_dim, name="encoded")(x)

        # Decoder layer
        x = Dense(np.prod(volume_size[1:]))(latent)
        x = Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)

        # Reverse on decoder
        for f in filters[::-1]:
            x = Conv2DTranspose(f, (3, 3), strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=channel_dim, name='dec_filter_'+str(f))(x)

        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid", name="decoded")(x)

        auto_encoder = Model(inputs, outputs, name="auto_encoder")
        return auto_encoder

In [6]:
def visualize_predictions(_decoded, gt, samples=10):
    outputs = None
    for i in range(0, samples):
        original = (gt[i] * 255).astype("uint8")
        recon = (_decoded[i] * 255).astype("uint8")
        output = np.hstack([original, recon])
        outputs = output if outputs is None else np.vstack([outputs, output])
    return outputs

In [7]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
IMAGE_SIZE = (256, 256)
checkpoint_path = "training_1_40_subclass/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [9]:
print("[INFO] indexing file images BreaKHis dataset...")
base_dataset = "subclass_scenario"
magnification = "40X"
class_dir = ['tubular_adenoma', 'phyllodes_tumor', 'papillary_carcinoma',
             'mucinous_carcinoma', 'lobular_carcinoma', 'fibroadenoma',
             'ductal_carcinoma', 'adenosis'
             ]
type_dataset = ['val', 'train']
dataset_train = []
dataset_val = []
for type_set in type_dataset:
    for class_item in class_dir:
        cur_dir = os.path.join(base_dataset, type_set, magnification ,class_item)
        for file in os.listdir(cur_dir):
            if type_set == 'train':
                dataset_train.append(os.path.join(cur_dir, file))
            else:
                dataset_val.append(os.path.join(cur_dir, file))

[INFO] indexing file images BreaKHis dataset...


In [10]:
print("[INFO] load images BreaKHis dataset...")
#  load images
train_images = []
val_images = []
for type_set in type_dataset:
    cur_dataset = dataset_train if type_set == 'train' else dataset_val
    for image_path in cur_dataset:
        if ".png" in image_path:
            image = cv2.imread(image_path)
            image = cv2.resize(image, IMAGE_SIZE)
            if type_set == 'train':
                train_images.append(image)
            else:
                val_images.append(image)

[INFO] load images BreaKHis dataset...


In [11]:
# normalization
print("[INFO] normalization...")
train_x = np.array(train_images).astype("float32") / 255.0
val_x = np.array(val_images).astype("float32") / 255.0

[INFO] normalization...


In [12]:
print("[INFO] building auto encoder...")
auto_encoder = ConvAutoEncoder.build(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_loss",
    verbose = 1,
    mode='min',
    save_best_only=True)
auto_encoder.compile(loss="mse", optimizer=opt)
auto_encoder.summary()

[INFO] building auto encoder...
Model: "auto_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 128)     3584      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 128)     0         
_________________________________________________________________
enc_filter_128 (BatchNormali (None, 128, 128, 128)     512       
_________________________________________________________________
flatten (Flatten)            (None, 2097152)           0         
_________________________________________________________________
encoded (Dense)              (None, 48)                100663344 
_________________________________________________________________
dense (Dense)         

In [15]:
# train the convolutional auto encoder
print("[INFO] training auto encoder...")
H = auto_encoder.fit(
    train_x, train_x,
    shuffle=True,
    validation_data=(val_x, val_x),
    epochs=EPOCHS,
    callbacks = [checkpoint_callback],
    batch_size=16)

[INFO] training auto encoder...
Epoch 1/50
102/102 [==============================] - 37s 351ms/step - loss: 0.0399 - val_loss: 0.0156

Epoch 00001: val_loss improved from inf to 0.01561, saving model to training_1_40_subclass/cp.ckpt
Epoch 2/50
102/102 [==============================] - 36s 353ms/step - loss: 0.0110 - val_loss: 0.0195

Epoch 00002: val_loss did not improve from 0.01561
Epoch 3/50
102/102 [==============================] - 36s 353ms/step - loss: 0.0106 - val_loss: 0.0235

Epoch 00003: val_loss did not improve from 0.01561
Epoch 4/50
102/102 [==============================] - 36s 353ms/step - loss: 0.0103 - val_loss: 0.0209

Epoch 00004: val_loss did not improve from 0.01561
Epoch 5/50
102/102 [==============================] - 36s 356ms/step - loss: 0.0102 - val_loss: 0.0187

Epoch 00005: val_loss did not improve from 0.01561
Epoch 6/50
102/102 [==============================] - 36s 357ms/step - loss: 0.0101 - val_loss: 0.0181

Epoch 00006: val_loss did not improve fro

In [ ]:
print("[INFO] making predictions...")
decoded = auto_encoder.predict(val_x)
vis = visualize_predictions(decoded, val_x)
image_example_path = "reconstruction"
cv2.imwrite(os.path.join(image_example_path,"recon_vis_40_subclass.png"), vis)
N = np.arange(0, EPOCHS)
plt.figure(figsize=(15,7))
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [18]:
with open('training_1_40_subclass.json', 'w') as f:
    json.dump(H.history, f)

In [19]:
auto_encoder.save('training_1_40_subclass.h5')